# Globoko učenje, vaje 1
## 1. Nevronske mreže s knjižnico pytorch

### 1.1 Instalacija
PyTorch je trenutno verjetno najpopularnejša knjižnica za delo z nevronskimi mrežami. Za instalacijo najprej aktivirajmo pythonovo okolje, ki ga uporabljamo za vaje (navigiramo v mapo okolja, podmapa Scripts, ter v ukazni vrstici kličemo **activate**). Instalirajmo osnoven paket pytorcha ter različico za grafovske nevronske mreže: **pip install torch torch-geometric**.

### 1.2 Nalaganje podatkov
Vajam so priložene datoteke s podatki o znanstvenih člankih. Vsak od njih je opisan s frekvencami posameznih besed (bag-of-words pristop), ki so podane v datoteki **podatki1_x.txt**. Vsak članek je kategoriziran v eno izmed sedmih znanstvenih področij. Razredi so podani v **podatki1_y.txt**. Poleg tega so podatki že razdeljeni v učno in testno množico. Datoteka "podatki1_train_mask.txt" podaja binarne vrednosti, ki povedo, ali je primer v učni množici.

Naloži vse tri, najlažje z uporabo **np.loadtxt**, ter si jih oglej.

In [7]:
import numpy as np

X = np.loadtxt('podatki1_x.txt')
y = np.loadtxt('podatki1_y.txt')
train = np.loadtxt('podatki1_train_mask.txt')

print(train)


[1. 1. 1. ... 0. 0. 0.]


Pytorch za predstavitev podatkov uporablja svoj razred Tensor (**torch.tensor**), v katerega moramo pretvoriti svoje podatke. Prav tako moramo zagotoviti, da so podatki pravega tipa - značilke so realna števila (**float**), razredi so celoštevilski (**long**), učna in testna maska pa sta binarni (**bool**). Poskrbi za ustrezno pretvorbo podatkov.

In [12]:
import torch

x_data = torch.tensor(X).float()
y_data = torch.tensor(y).long()
train_mask = torch.tensor(train).bool()
test_mask = torch.tensor(np.logical_not(train)).bool() #pomagaš si lahko z np.logical_not


### 1.3 Sestavljanje nevronske mreže

Mrežo implementiramo kot razred, ki deduje po **torch.nn.Module**. Definirati mora metodi **__init__**, v kateri inicializiramo vse gradnike, ter **forward** ki opisuje potek izračuna v naši mreži. Ključni elementi:
- Linear: polno povezana plast, osnovni gradnik navadnih nevronskih mrež. Podati ji moramo dimenzije vhoda in izhoda, pri čemer se mora izhod n-te plasti ujemati z vhodom n+1-te plasti. Vhod prve plasti je število značilk, izhod zadnje plasti pa število razredov.
- relu: Rectified Linear Unit - aktivacijska funkcija, ki mreži daje nelinearnost. Aktivacijske funkcije postavljamo za posamezne plasti, pomembno pa je, da je ne postavimo pred izhod, saj bi nam pokvarila napovedi.
- dropout: element, ki pri treniranju zavrže del nevronov v posamezni plasti (delež p), s čimer se borimo proti preprileganju.

Dopolni definicijo nevronske mreže!

In [24]:
print(x_data.shape[1])
size_input = x_data.shape[1]
size_output = len(y_data.unique())
print(size_output)

1433
7


In [29]:
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self, n_skritih):
        super().__init__()
        torch.manual_seed(12345)

        self.lin1 = Linear(size_input, n_skritih)
        self.lin2 = Linear(n_skritih, n_skritih)
        self.lin3 = Linear(n_skritih, size_output)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin3(x)
        return x


### 1.4 Treniranje nevronske mreže

V knjižnici pytorch moramo učno zanko napisati sami. Za začetek potrebujemo naslednje elemente:
- model (instanca mreže, ki smo jo definirali zgoraj, priporočeno število nevronov v skriti plasti: 16)
- kriterijska funkcija, ki na podlagi napovedi mreže in pravih vrednosti izračuna napako. Uporabili bomo prečno entropijo (**torch.nn.CrossEntropyLoss**).
- optimizator. Uporabili bomo Adam (**torch.optim.Adam**). Podati mu moramo parametre mreže, ki jih dobimo z **model.parameters()**, dobro pa je definirati tudi hitrost učenja (npr. **lr=0.01**) in parameter L2 regularizacije (npr. **weight_decay=0.0005**). 

In [30]:
model = MLP(n_skritih=16)
kriterijska_funkcija = torch.nn.CrossEntropyLoss()
optimizator = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)

Koraki učne zanke so sledeči:
1. s klicem funkcije **optimizator.zero_grad()** postavimo odvode na nič
2. učne podatke pošljemo skozi mrežo (**model(podatki)**) ter si shranimo rezultat
3. na podlagi rezultata in pravih vrednosti izračunamo napako s kriterijsko funkcijo
4. izračunamo odvode s klicem **napaka.backward()** 
5. posodobimo parametre mreže na podlagi odvodov s klicem **optimizator.step()**

Pomembno je še, da pred začetkom treniranja modelu povemo, da je čas za treniranje s klicem model.train(). Razlog je, da se dropout uporablja samo med treniranjem, ne pa med evaluacijo in uporabo mreže.

Dopolni!

In [31]:
def train():
      optimizator.zero_grad()
      x = model(x_data[train_mask])
      napaka = kriterijska_funkcija(x, y_data[train_mask])
      napaka.backward()
      optimizator.step()
      return napaka

model.train()
for epoch in range(1, 201):
    loss = train()
    if epoch%10 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 010, Loss: 1.6781
Epoch: 020, Loss: 1.2324
Epoch: 030, Loss: 0.9818
Epoch: 040, Loss: 0.7924
Epoch: 050, Loss: 0.7121
Epoch: 060, Loss: 0.6555
Epoch: 070, Loss: 0.5943
Epoch: 080, Loss: 0.5877
Epoch: 090, Loss: 0.5527
Epoch: 100, Loss: 0.4642
Epoch: 110, Loss: 0.5736
Epoch: 120, Loss: 0.5858
Epoch: 130, Loss: 0.4499
Epoch: 140, Loss: 0.5542
Epoch: 150, Loss: 0.4564
Epoch: 160, Loss: 0.4252
Epoch: 170, Loss: 0.5871
Epoch: 180, Loss: 0.4108
Epoch: 190, Loss: 0.5036
Epoch: 200, Loss: 0.5407


### 1.5 Evaluacija nevronske mreže

Ko smo mrežo natrenirali, želimo izračunati še njeno napako na testni množici. Pri tem:
- prej mreži povemo, da je čas za evaluacijo s klicem **model.eval()**,
- pošljemo testne podatke skozi mrežo in si shranimo rezultat

Oglej si izhod iz mreže in premisli, kaj pomeni. Potem izračunaj testno natačnost. V pomoč ti bo funkcija **argmax(dim=1)** rezultata.

In [43]:
from sklearn.metrics import accuracy_score

model.eval()
out = model(x_data[test_mask])
y_pred = out.argmax(dim=1)
y_true = y_data[test_mask]
acc = accuracy_score(y_true, y_pred)

print(f'Testna natancnost: {acc:.4f}')

Testna natancnost: 0.4942


Kako dobra je natančnost mreže? Za primerjavo izračunaj še testno napako modela, ki vedno napove najpogostejši razred, ali pa naključnega.

In [44]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_data[train_mask], y_data[train_mask])
dummy_pred = dummy_clf.predict(x_data[test_mask])
dummy_acc = accuracy_score(y_data[test_mask], dummy_pred)
print(dummy_acc)

0.12889408099688474


## 2. Grafovske nevronske mreže s knjižnico torch-geometric

Za grafovske nevronske mreže bomo uprabili paket **torch-geometric**, krajše imenovan pyG. Delali bomo z istimi podatki o znanstvenih člankih, vendar bomo tokrat uporabili še informacijo o citatih, ki je podana v obliki grafa. Vsak članek je vozlišče na grafu, vozlišča pa sta povezana, če je eden izmed člankov citiral drugega. Ideja je, so članki, ki se citirajo, verjetno iz istega področja. 

### 2.1 Nalaganje podatkov

Uporabili bomo iste podatke kot prej, le da tokrat potrebujemo še **podatki1_povezave.txt**, ki ga lahko prav tako naložiš z **np.loadtxt**. Oglej si, v kakšni obliki so povezave podane, potem pa jih spremeni v torch.tensor celoštevilskega tipa.

In [48]:
povezave = np.loadtxt('podatki1_povezave.txt')
povezave = torch.tensor(povezave).long()
print(povezave)

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


Zdaj sicer imamo vse kar potrebujemo, ampak za ilustracijo poglejmo še pyG-jev objekt Data, ki nam lahko pove kup uporabnih informacij o grafu.

In [49]:
from torch_geometric.data import Data

data = Data(x=x_data, y=y_data, edge_index=povezave)

print(f'Število vozlišč: {data.num_nodes}')
print(f'Število povezav: {data.num_edges}')
print(f'Povprečno število povezav na vozlišče: {data.num_edges / data.num_nodes:.2f}')
print(f'Izolirana vozlišča: {data.has_isolated_nodes()}')
print(f'Self-loops: {data.has_self_loops()}')
print(f'Neusmerjen: {data.is_undirected()}')

Število vozlišč: 2708
Število povezav: 10556
Povprečno število povezav na vozlišče: 3.90
Izolirana vozlišča: False
Self-loops: False
Neusmerjen: True


### 2.2 Sestavljanje grafovske nevronske mreže

Definicija mreže bo zelo podobna tisti v 1.3. Namesto polno povezanih linearnih plasti bomo uporabili grafovske konvolucijske plasti **GCNConv**, ki jih inicializiramo enako kot prej. Razlika pa je, da jim pri izračunu poleg značilk podamo tudi povezave grafa.

Dopolni!

In [69]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, n_skritih):
        super().__init__()
        torch.manual_seed(0)
        self.conv1 = GCNConv(size_input, n_skritih)
        self.conv2 = GCNConv(n_skritih, n_skritih)
        self.conv3 = GCNConv(n_skritih, size_output)
        

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        return x

### 2.3 Treniranje grafovske nevronske mreže

Koda za naše grafovske nevronske mreže je skoraj enaka kot pri navadni nevronski mreži, le da mreži podamo še graf. Pomembna razlika pa je, da moramo tokrat modelu podati vse podatke za izračun, ne samo učne množice, sicer ne more uporabiti celotnega grafa in vrže napako. Seveda pa pri izračunu napake upoštevamo samo učno množico.

Natreniraj svojo grafovsko nevronsko mrežo!

### 2.3 Treniranje grafovske nevronske mreže

In [70]:
model = GCN(n_skritih=16)
kriterijska_funkcija = torch.nn.CrossEntropyLoss()
optimizator = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)

def train():
      optimizator.zero_grad()  
      rezultat = model(x_data, povezave) 
      napaka = kriterijska_funkcija(rezultat[train_mask], y_data[train_mask]) 
      napaka.backward()  
      optimizator.step()  
      return napaka

model.train()
for epoch in range(1, 201):
    loss = train()
    if epoch%10 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 010, Loss: 1.1551
Epoch: 020, Loss: 0.4449
Epoch: 030, Loss: 0.1861
Epoch: 040, Loss: 0.0620
Epoch: 050, Loss: 0.0742
Epoch: 060, Loss: 0.0569
Epoch: 070, Loss: 0.0756
Epoch: 080, Loss: 0.1085
Epoch: 090, Loss: 0.0591
Epoch: 100, Loss: 0.0334
Epoch: 110, Loss: 0.0567
Epoch: 120, Loss: 0.0565
Epoch: 130, Loss: 0.0536
Epoch: 140, Loss: 0.0680
Epoch: 150, Loss: 0.0435
Epoch: 160, Loss: 0.0356
Epoch: 170, Loss: 0.0305
Epoch: 180, Loss: 0.0276
Epoch: 190, Loss: 0.0680
Epoch: 200, Loss: 0.0261


### 2.4 Evaluacija grafovske nevronske mreže

Izračunaj testno napako nove mreže podobno kot prej. Je informacija o citatih izboljšala rezultat?

In [71]:

model.eval()
gout = model(x_data, povezave)
y_pred_g = gout.argmax(dim=1)
y_true = y_data[test_mask]
testna_natancnost = accuracy_score(y_true, y_pred_g[test_mask])

print(f'Test Accuracy: {testna_natancnost:.4f}')

Test Accuracy: 0.7831


## 3 Arhitekture mrež in hiperparametri

Arhitekura mreže ter razni hiperparametri imajo velik vpliv na delovanje. Pogosto jih moramo optimizirati v zahtevnih računskih eksperimentih, podobno kot smo to počeli v DN1. Da dobiš občutek, kaj se dogaja, se vrni k obema mrežama ter poskusi:
- dodati tretjo linearno plast v navadno nevronsko mrežo,
- dodati tretjo konvolucijsko plast v grafovsko nevronsko mrežo,
- dodati linearno plast na konec grafovske nevronske mreže.

Potem preizkusi še vpliv parametrov:
- število nevronov v linearni ali konvolucijski plasti
- verjetnost v dropout plasti
- izbira aktivacijske funkcije (https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity)
- parameter hitrosti učenja v optimizatorju
- parameter L2 regularizacije v optimizatorju
- število epohov

## Dodatno

Današnje vaje so bile prirejene iz drugega učnega zvezka pyG-jeve dokumentacije: https://pytorch-geometric.readthedocs.io/en/latest/get_started/colabs.html

Napovedovali smo kategorije posameznih vozlišč v grafu. Drugačen tip problema, ki se pogosto rešuje z grafovskimi nevronskimi mrežami, je napovedovanje kategorije celotnega grafa, pri čemer so naši podatki sestavljeni iz množice grafov. Primer je napovedovanje lastosti molekul, ki jih lahko opišemo kot povezave med atomi (vozlišči). Reševanje takega problema naslavlja tretji zvezek v zgornji dokumentaciji.

Če te tematika zanima, lahko nadaljuješ s pyG-jevimi učnimi zvezki.